<a href="https://colab.research.google.com/github/thanaprondtn/bsc_dpdm24/blob/main/midterm_dpdm24_570_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [375]:
import pandas as pd
import numpy as np

In [376]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [377]:
import os
os.chdir('/content/drive/MyDrive/MIDterm')

!pwd # เช็ค path ที่กำลังทำงานอยู่

/content/drive/MyDrive/MIDterm


#เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง  

In [378]:
data_house = pd.read_excel("/content/drive/MyDrive/MIDterm/Data House.xlsx")

In [379]:
# เลือกเฉพาะ columns ที่ต้องการ
selected_columns = ['id','renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']
data_house = data_house[selected_columns]

In [380]:
# ตรวจสอบ missing values
data_house.isnull().sum()

,0
id,0
renovate,211
number_of_rai,0
number_of_ngan,0
number_of_va,0
number_of_story,549
number_of_bedroom,27
number_of_bathroom,0
direction,0
furniture,0


In [381]:
# prompt: แก้ missing values ของ data_house

# Fill missing values in numerical columns with the mean
numerical_cols = ['number_of_rai', 'number_of_ngan', 'number_of_va', 'number_of_story', 'number_of_bedroom', 'number_of_bathroom', 'usable_area', 'number_of_parking', 'sell_price']
for col in numerical_cols:
    data_house[col] = data_house[col].fillna(data_house[col].mean())

# Fill missing values in categorical columns with the mode
categorical_cols = ['renovate', 'direction', 'furniture', 'district', 'sub_district', 'zone']
for col in categorical_cols:
    data_house[col] = data_house[col].fillna(data_house[col].mode()[0])

#Check for any remaining missing values
print(data_house.isnull().sum())

id                    0
renovate              0
number_of_rai         0
number_of_ngan        0
number_of_va          0
number_of_story       0
number_of_bedroom     0
number_of_bathroom    0
direction             0
furniture             0
usable_area           0
number_of_parking     0
sell_price            0
district              0
sub_district          0
zone                  0
dtype: int64


In [382]:
# prompt: ตรวจหา Outliers ของ ทุก columns ใน data_house และสรุปว่า column ใดบ้างที่มี Outliers

def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

columns_with_outliers = []
for col in data_house.columns:
  if pd.api.types.is_numeric_dtype(data_house[col]):
    outliers = detect_outliers(data_house,col)
    if not outliers.empty:
        columns_with_outliers.append(col)
        print(f"Outliers found in column: {col}")
        print(outliers)
    else:
        print(f"No outliers found in column: {col}")

print("\nColumns with outliers:")
columns_with_outliers

Outliers found in column: number_of_rai
          id renovate  number_of_rai  number_of_ngan  number_of_va  \
2         16       no              1               0           8.0   
20       510       no              4               1          66.0   
25       525       no              1               0           0.0   
34       535       no              3               2          92.0   
35       537       no              7               3          39.8   
...      ...      ...            ...             ...           ...   
1547  30157P       no             25               0           0.0   
1566  30198P       no              1               1         101.0   
1574  30212P       no              7               0          43.0   
1623  30305P       no              1               1           1.0   
1643  20449P       no              1              95         780.0   

      number_of_story  number_of_bedroom  number_of_bathroom direction  \
2            1.000000          11.000000     

['number_of_rai',
 'number_of_ngan',
 'number_of_va',
 'number_of_story',
 'number_of_bedroom',
 'number_of_bathroom',
 'usable_area',
 'number_of_parking',
 'sell_price']

In [383]:
# prompt: จัดการกับ outliers ของ ทุก columns ใน data_house

def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with the bounds
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df


for col in data_house.columns:
    if pd.api.types.is_numeric_dtype(data_house[col]):
        data_house = handle_outliers(data_house, col)

# Verification (optional): Check if outliers still exist
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

for col in data_house.columns:
    if pd.api.types.is_numeric_dtype(data_house[col]):
        outliers = detect_outliers(data_house, col)
        if not outliers.empty:
            print(f"Outliers still found in column: {col}")
            print(outliers)
        else:
            print(f"No outliers found in column: {col}")

No outliers found in column: number_of_rai
No outliers found in column: number_of_ngan
No outliers found in column: number_of_va
No outliers found in column: number_of_story
No outliers found in column: number_of_bedroom
No outliers found in column: number_of_bathroom
No outliers found in column: usable_area
No outliers found in column: number_of_parking
No outliers found in column: sell_price


In [384]:
data_house.head()

,id,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,1,no,0.0,0.0,45.0,2.0,2.0,2.0,ไม่ระบุ,ไม่มี,244.000000,2.0,390000.0,ชนบท,ศรีบุญเรือง,กลางเมือง-เมืองเก่า-กุดกว้าง
1,4,no,0.0,0.0,88.0,1.0,1.0,1.0,ไม่ระบุ,ไม่มี,164.000000,3.0,820000.0,น้ำพอง,ทรายมูล,กลางเมือง-เมืองเก่า-กุดกว้าง
2,16,no,0.0,0.0,8.0,1.0,7.5,3.5,ไม่ระบุ,ไม่ระบุ,440.162058,5.0,3260000.0,มัญจาคีรี,กุดเค้า,กลางเมือง-เมืองเก่า-กุดกว้าง
3,18,no,0.0,0.0,61.6,2.0,3.0,3.0,ไม่ระบุ,ไม่ระบุ,203.000000,2.0,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,19,no,0.0,0.0,72.3,2.0,3.0,2.0,ไม่ระบุ,ไม่ระบุ,164.000000,2.0,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน


In [385]:
# prompt: ทำ Feature Engineering ที่เหมาะสม กับ data_house  เพื่อปรับปรุงคุณภาพของข้อมูล บอกเหตุผลที่ใช้วิธีนั้นด้วย

# Feature Engineering

# 1. Combine area features: Create a total area feature
data_house['total_area'] = data_house['number_of_rai'] * 400 + data_house['number_of_ngan'] * 100 + data_house['number_of_va']
#Reason: Combining rai, ngan, and va into a single 'total_area' feature simplifies the representation of land size, making it easier for models to interpret and potentially improving predictive accuracy.

# 2.  Create a new feature for total rooms (bedrooms + bathrooms)
data_house['total_rooms'] = data_house['number_of_bedroom'] + data_house['number_of_bathroom']
#Reason: The total number of rooms might be a better indicator of house size than bedrooms or bathrooms alone.


# 3.  Handle Categorical Features: One-Hot Encoding or Label Encoding (Choose based on feature cardinality)
# Use pd.get_dummies for One-Hot Encoding if you don't have many categories for a feature.
# If a feature has many categories, consider using Label Encoding (e.g., from sklearn.preprocessing) to reduce dimensionality
data_house = pd.get_dummies(data_house, columns=['direction', 'furniture', 'district', 'sub_district', 'zone', 'renovate'], drop_first=True)
#Reason:  One-hot encoding converts categorical variables into numerical representations that machine learning models can understand.  'drop_first=True' helps prevent multicollinearity (redundant features).  However, using label encoding may be more suitable if you have a lot of categories in a certain variable.

# 4.  Feature Scaling (Normalization or Standardization) for Numerical Features (optional but recommended)
from sklearn.preprocessing import MinMaxScaler

# Numerical columns to scale
numerical_cols_to_scale = ['total_area', 'number_of_story', 'number_of_bedroom', 'number_of_bathroom', 'usable_area', 'number_of_parking', 'sell_price', 'total_rooms']  # Include your new total_area and other numerical features

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the numerical features
data_house[numerical_cols_to_scale] = scaler.fit_transform(data_house[numerical_cols_to_scale])
#Reason: Scaling numerical features ensures that features with larger values don't dominate the model's learning process, improving model performance.


In [386]:
data_house.head()

,id,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,usable_area,number_of_parking,sell_price,...,zone_ศิลา-โกทา-หนองไผ่-บ้านดอน,zone_สามเหลี่ยม-กังสดาล-ม.ขอนแก่น,zone_เหล่านาดี-บ้านสะอาด-บ้านโจด,zone_แดงใหญ่,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,renovate_yes,renovate_ไม่ระบุ
0,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,False,False,False,False,False,False
1,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.6,0.102524,...,False,False,False,False,False,False,False,False,False,False
2,16,0.0,0.0,8.0,0.285714,1.000000,1.0,1.000000,1.0,0.409466,...,False,False,False,False,False,False,False,False,False,False
3,18,0.0,0.0,61.6,0.571429,0.400000,0.8,0.419090,0.4,0.628351,...,False,False,False,False,False,False,False,False,False,False
4,19,0.0,0.0,72.3,0.571429,0.400000,0.4,0.323562,0.4,0.439657,...,False,False,False,False,False,False,False,False,False,False


#เตรียมข้อมูลเพื่อทำนายราคาประเมินบ้าน

In [387]:
import json
import geopy.distance

# อ่านข้อมูลจากไฟล์ JSON
with open("/content/drive/MyDrive/MIDterm/7-eleven.json", "r") as f:
    data_711 = json.load(f)
with open("/content/drive/MyDrive/MIDterm/school.json", "r") as f:
    data_school = json.load(f)

# สร้าง function สำหรับคำนวณระยะห่างระหว่างพิกัด
import geopy.distance

def calculate_distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return geopy.distance.geodesic(coords_1, coords_2).km

# เพิ่ม columns 'd1000-7/11' และ 'd1000-school' ใน data_house
data_house['d1000-7/11'] = 0
data_house['d1000-school'] = 0

# คำนวณจำนวน 7-11 และ โรงเรียนในรัศมี 1 กิโลเมตร
for index, row in data_house.iterrows():
    for item in data_711:
        # Check if item is a dictionary before accessing 'id'
        if isinstance(item, dict):
            item_id = int(item['id']) if isinstance(item['id'], str) else item['id']

            if item_id == row['id']:
                for store in item['nearby']:
                    distance = calculate_distance(row['latitude'], row['longitude'], store['latitude'], store['longitude'])
                    if distance <= 1:
                        data_house.loc[index, 'd1000-7/11'] += 1
    for item in data_school:
        # Check if item is a dictionary before accessing 'id'
        if isinstance(item, dict):
            item_id = int(item['id']) if isinstance(item['id'], str) else item['id']

            if item_id == row['id']:
                for school in item['nearby']:
                    distance = calculate_distance(row['latitude'], row['longitude'], school['latitude'], school['longitude'])
                    if distance <= 1:
                        data_house.loc[index, 'd1000-school'] += 1



# คำนวณจำนวน 7-11 และ โรงเรียนในรัศมี 1 กิโลเมตร และจัดการกรณีที่ไม่พบในรัศมี 5 กิโลเมตร
for index, row in data_house.iterrows():
    found_711_within_5km = False  # Flag to track if 7-11 found within 5km
    found_school_within_5km = False # Flag to track if school found within 5km

    # ตรวจสอบข้อมูล 7-11 ใน data_711
    for item in data_711:
        # ตรวจสอบว่า item เป็น dictionary และมี id ตรงกับบ้านใน data_house หรือไม่
        if isinstance(item, dict) and item.get('id') == row['id']:
            # ถ้าเจอบ้านที่มี id ตรงกัน ให้วนลูปตรวจสอบ 7-11 ใกล้เคียง
            for store in item['nearby']:
                distance = calculate_distance(row['latitude'], row['longitude'], store['latitude'], store['longitude'])
                # นับจำนวน 7-11 ในรัศมี 1 กิโลเมตร
                if distance <= 1:
                    data_house.loc[index, 'd1000-7/11'] += 1
                # ตั้งค่า Flag ถ้าพบ 7-11 ในรัศมี 5 กิโลเมตร
                if distance <= 5:
                    found_711_within_5km = True

    # ตรวจสอบข้อมูลโรงเรียนใน data_school (logic คล้ายกับการตรวจสอบ 7-11)
    for item in data_school:
        if isinstance(item, dict) and item.get('id') == row['id']:
            for school in item['nearby']:
                distance = calculate_distance(row['latitude'], row['longitude'], school['latitude'], school['longitude'])
                if distance <= 1:
                    data_house.loc[index, 'd1000-school'] += 1
                if distance <= 5:
                    found_school_within_5km = True

    # จัดการกรณีที่ไม่พบข้อมูลในรัศมี 5 กิโลเมตร
    if not found_711_within_5km:
        data_house.loc[index, 'd1000-7/11'] = -1  # หรือค่าอื่นๆ ที่คุณต้องการใช้
    if not found_school_within_5km:
        data_house.loc[index, 'd1000-school'] = -1 # หรือค่าอื่นๆ ที่คุณต้องการใช้

In [388]:
data_house

,id,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,usable_area,number_of_parking,sell_price,...,zone_เหล่านาดี-บ้านสะอาด-บ้านโจด,zone_แดงใหญ่,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,renovate_yes,renovate_ไม่ระบุ,d1000-7/11,d1000-school
0,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.400000,0.048431,...,False,False,False,False,False,False,False,False,-1,-1
1,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.600000,0.102524,...,False,False,False,False,False,False,False,False,-1,-1
2,16,0.0,0.0,8.0,0.285714,1.000000,1.0,1.000000,1.000000,0.409466,...,False,False,False,False,False,False,False,False,-1,-1
3,18,0.0,0.0,61.6,0.571429,0.400000,0.8,0.419090,0.400000,0.628351,...,False,False,False,False,False,False,False,False,-1,-1
4,19,0.0,0.0,72.3,0.571429,0.400000,0.4,0.323562,0.400000,0.439657,...,False,False,False,False,False,False,False,False,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,0.0,0.0,73.0,0.285714,0.266667,0.0,0.188256,0.186954,0.054394,...,False,False,False,False,False,False,False,False,-1,-1
1652,20462P,0.0,0.0,84.0,0.285714,0.266667,0.0,0.213338,0.186954,0.089265,...,False,False,False,False,False,False,False,False,-1,-1
1653,20463P,0.0,0.0,45.0,0.571429,0.133333,0.0,0.383573,0.186954,0.056105,...,False,False,False,False,False,False,False,False,-1,-1
1654,20466P,0.0,0.0,59.0,0.571429,0.133333,0.0,0.428887,0.186954,0.113845,...,False,False,False,False,False,False,False,False,-1,-1


#เตรียมข้อมูลเพื่อทำนายคะแนนทรัพย์

In [389]:
# อ่านข้อมูลจากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน
appraiser_data = pd.read_excel("/content/drive/MyDrive/MIDterm/คำตอบนักประเมิน(ID).xlsx")

In [390]:
appraiser_data.head()

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [391]:
# prompt: เปลี่ยน ID เป็น id เปลี่ยนประเมินราคาซื้อขาย เป็น estimator และ คะแนนทรัพย์ เป็น score


# Rename columns
appraiser_data = appraiser_data.rename(columns={'ID': 'id', 'ประเมินราคาซื้อ-ขาย': 'estimator', 'ให้คะแนนทรัพย์': 'score'})

# Merge the appraiser data with the main dataframe
# Assuming 'id' is the common column for merging
data_house = pd.merge(data_house, appraiser_data[['id', 'estimator', 'score']], on='id', how='left')



In [392]:
data_house

,id,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,usable_area,number_of_parking,sell_price,...,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,renovate_yes,renovate_ไม่ระบุ,d1000-7/11,d1000-school,estimator,score
0,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.400000,0.048431,...,False,False,False,False,False,False,-1,-1,600000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
1,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.400000,0.048431,...,False,False,False,False,False,False,-1,-1,1300000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.600000,0.102524,...,False,False,False,False,False,False,-1,-1,750000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
3,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.600000,0.102524,...,False,False,False,False,False,False,-1,-1,1000000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
4,16,0.0,0.0,8.0,0.285714,1.000000,1.0,1.000000,1.000000,0.409466,...,False,False,False,False,False,False,-1,-1,2800000.0,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,20460P,0.0,0.0,73.0,0.285714,0.266667,0.0,0.188256,0.186954,0.054394,...,False,False,False,False,False,False,-1,-1,NaN,NaN
2367,20462P,0.0,0.0,84.0,0.285714,0.266667,0.0,0.213338,0.186954,0.089265,...,False,False,False,False,False,False,-1,-1,NaN,NaN
2368,20463P,0.0,0.0,45.0,0.571429,0.133333,0.0,0.383573,0.186954,0.056105,...,False,False,False,False,False,False,-1,-1,NaN,NaN
2369,20466P,0.0,0.0,59.0,0.571429,0.133333,0.0,0.428887,0.186954,0.113845,...,False,False,False,False,False,False,-1,-1,NaN,NaN


In [393]:
# prompt: ลบ ประเมินราคาซื้อ-ขาย ให้คะแนนทรัพย์

# Remove 'ประเมินราคาซื้อ-ขาย' and 'ให้คะแนนทรัพย์' columns if they exist
if 'ประเมินราคาซื้อ-ขาย' in data_house.columns:
    data_house = data_house.drop(columns=['ประเมินราคาซื้อ-ขาย'])
if 'ให้คะแนนทรัพย์' in data_house.columns:
    data_house = data_house.drop(columns=['ให้คะแนนทรัพย์'])


data_house

,id,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,usable_area,number_of_parking,sell_price,...,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,renovate_yes,renovate_ไม่ระบุ,d1000-7/11,d1000-school,estimator,score
0,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.400000,0.048431,...,False,False,False,False,False,False,-1,-1,600000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
1,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.400000,0.048431,...,False,False,False,False,False,False,-1,-1,1300000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.600000,0.102524,...,False,False,False,False,False,False,-1,-1,750000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
3,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.600000,0.102524,...,False,False,False,False,False,False,-1,-1,1000000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
4,16,0.0,0.0,8.0,0.285714,1.000000,1.0,1.000000,1.000000,0.409466,...,False,False,False,False,False,False,-1,-1,2800000.0,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,20460P,0.0,0.0,73.0,0.285714,0.266667,0.0,0.188256,0.186954,0.054394,...,False,False,False,False,False,False,-1,-1,NaN,NaN
2367,20462P,0.0,0.0,84.0,0.285714,0.266667,0.0,0.213338,0.186954,0.089265,...,False,False,False,False,False,False,-1,-1,NaN,NaN
2368,20463P,0.0,0.0,45.0,0.571429,0.133333,0.0,0.383573,0.186954,0.056105,...,False,False,False,False,False,False,-1,-1,NaN,NaN
2369,20466P,0.0,0.0,59.0,0.571429,0.133333,0.0,0.428887,0.186954,0.113845,...,False,False,False,False,False,False,-1,-1,NaN,NaN


In [394]:
# prompt: score เอาแค่ตัวอักษรตัวแรก นอกนั้นลบเลย

# Assuming 'score' column exists in your DataFrame 'data_house'

# Extract the first letter of the 'score' column
data_house['score'] = data_house['score'].astype(str).str[0]

data_house

,id,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,usable_area,number_of_parking,sell_price,...,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,renovate_yes,renovate_ไม่ระบุ,d1000-7/11,d1000-school,estimator,score
0,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.400000,0.048431,...,False,False,False,False,False,False,-1,-1,600000.0,4
1,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.400000,0.048431,...,False,False,False,False,False,False,-1,-1,1300000.0,4
2,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.600000,0.102524,...,False,False,False,False,False,False,-1,-1,750000.0,3
3,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.600000,0.102524,...,False,False,False,False,False,False,-1,-1,1000000.0,4
4,16,0.0,0.0,8.0,0.285714,1.000000,1.0,1.000000,1.000000,0.409466,...,False,False,False,False,False,False,-1,-1,2800000.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,20460P,0.0,0.0,73.0,0.285714,0.266667,0.0,0.188256,0.186954,0.054394,...,False,False,False,False,False,False,-1,-1,NaN,n
2367,20462P,0.0,0.0,84.0,0.285714,0.266667,0.0,0.213338,0.186954,0.089265,...,False,False,False,False,False,False,-1,-1,NaN,n
2368,20463P,0.0,0.0,45.0,0.571429,0.133333,0.0,0.383573,0.186954,0.056105,...,False,False,False,False,False,False,-1,-1,NaN,n
2369,20466P,0.0,0.0,59.0,0.571429,0.133333,0.0,0.428887,0.186954,0.113845,...,False,False,False,False,False,False,-1,-1,NaN,n


In [395]:
# prompt: จัดการกับ Missing values, Outliers, Normalization ของ data_house

# Assuming the code you provided is already executed and 'data_house' DataFrame is available.

# Further handling of missing values (if any remain after previous steps)
# You might want to use more advanced imputation techniques if needed, like KNN imputation.
# For now, I'll use the previous method as it's already present in your code.


#Normalization of Numerical Features (if not already done)
# You have already included the MinMaxScaler in your code.

# Outlier handling (already done in the provided code).
# If you want to explore other methods, consider RobustScaler or other outlier removal techniques.

# Additional Feature Engineering (example):
# Create interaction features if applicable (e.g., multiplying relevant numerical features).

# Example interaction feature: usable_area * number_of_bedroom
# data_house['area_bedroom_interaction'] = data_house['usable_area'] * data_house['number_of_bedroom']


# Convert 'score' column to numeric, coercing errors to NaN, then fill with the median
data_house['score'] = pd.to_numeric(data_house['score'], errors='coerce')
data_house['score'] = data_house['score'].fillna(data_house['score'].median())

# Final Check for missing values after all the processing
print(data_house.isnull().sum())

# Display the processed DataFrame
data_house.head()

id                    0
number_of_rai         0
number_of_ngan        0
number_of_va          0
number_of_story       0
                   ... 
renovate_ไม่ระบุ      0
d1000-7/11            0
d1000-school          0
estimator           589
score                 0
Length: 209, dtype: int64


,id,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,usable_area,number_of_parking,sell_price,...,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,renovate_yes,renovate_ไม่ระบุ,d1000-7/11,d1000-school,estimator,score
0,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,False,False,-1,-1,600000.0,4.0
1,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,False,False,-1,-1,1300000.0,4.0
2,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.6,0.102524,...,False,False,False,False,False,False,-1,-1,750000.0,3.0
3,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.6,0.102524,...,False,False,False,False,False,False,-1,-1,1000000.0,4.0
4,16,0.0,0.0,8.0,0.285714,1.000000,1.0,1.000000,1.0,0.409466,...,False,False,False,False,False,False,-1,-1,2800000.0,2.0


In [396]:
data_house.columns

Index(['id', 'number_of_rai', 'number_of_ngan', 'number_of_va',
       'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
       'usable_area', 'number_of_parking', 'sell_price',
       ...
       'zone_โคกสี', 'zone_โนนทัน-บึงแก่นนคร',
       'zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง',
       'zone_ในเมือง-รื่นรมย์-เทพารักษ์', 'renovate_yes', 'renovate_ไม่ระบุ',
       'd1000-7/11', 'd1000-school', 'estimator', 'score'],
      dtype='object', length=209)

In [397]:
# prompt: data_house columns ที่มีคำว่า zone ทุกอันรวมกัน และแยกชื่อของ zone มาเป้นค่าใน row เช่น zone_โคกสี ชื่อ column คือ zone ค่าคือ โคกสี

# Assuming data_house DataFrame is already created and processed as in the provided code.

zone_cols = [col for col in data_house.columns if 'zone_' in col]
zone_df = pd.DataFrame({'id': data_house['id']}) # Initialize zone_df with id for merging later

for col in zone_cols:
  zone_name = col.replace('zone_', '')
  zone_df[zone_name] = data_house[col] # Directly add zone columns instead of concatenating
  data_house = data_house.drop(col, axis=1)

# Melt the zone_df to get the desired format with 'zone' and 'value' columns
zone_df = zone_df.melt(id_vars=['id'], var_name='zone', value_name='value')
zone_df = zone_df[zone_df['value'] == 1]  # Filter to keep only the selected zones for each property

# Merge with the original dataframe
data_house = pd.merge(data_house, zone_df[['id', 'zone']], on='id', how='left')

In [398]:
data_house.head()

,id,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,usable_area,number_of_parking,sell_price,...,sub_district_ในเมือง,sub_district_ใหม่นาเพียง,sub_district_ไชยสอ,renovate_yes,renovate_ไม่ระบุ,d1000-7/11,d1000-school,estimator,score,zone
0,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,False,-1,-1,600000.0,4.0,NaN
1,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,False,-1,-1,1300000.0,4.0,NaN
2,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.6,0.102524,...,False,False,False,False,False,-1,-1,750000.0,3.0,NaN
3,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.6,0.102524,...,False,False,False,False,False,-1,-1,1000000.0,4.0,NaN
4,16,0.0,0.0,8.0,0.285714,1.000000,1.0,1.000000,1.0,0.409466,...,False,False,False,False,False,-1,-1,2800000.0,2.0,NaN


In [399]:
data_house.columns

Index(['id', 'number_of_rai', 'number_of_ngan', 'number_of_va',
       'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
       'usable_area', 'number_of_parking', 'sell_price',
       ...
       'sub_district_ในเมือง', 'sub_district_ใหม่นาเพียง',
       'sub_district_ไชยสอ', 'renovate_yes', 'renovate_ไม่ระบุ', 'd1000-7/11',
       'd1000-school', 'estimator', 'score', 'zone'],
      dtype='object', length=193)

In [400]:
# คำนวณราคาเฉลี่ยของแต่ละ zone
zone_avg_price = data_house.groupby('zone')['sell_price'].mean()

# Check if zone_avg_price is empty
if zone_avg_price.empty:
    print("zone_avg_price is empty. Check the 'zone' column in data_house.")
else:
    # หา zone ที่มีราคาเฉลี่ยสูงที่สุด
    highest_avg_price_zone = zone_avg_price.idxmax()

    # แสดงผลลัพธ์
    print(f"Zone ที่มีราคาซื้อขายบ้านเฉลี่ยสูงที่สุด: {highest_avg_price_zone}")

Zone ที่มีราคาซื้อขายบ้านเฉลี่ยสูงที่สุด: สามเหลี่ยม-กังสดาล-ม.ขอนแก่น


In [401]:
house_with_most_711 = data_house[data_house['d1000-7/11'] == data_house['d1000-7/11'].max()]
price_of_house_with_most_711 = house_with_most_711['sell_price'].iloc[0]
print(f"Price of the house with the most 7-11s within 1km: {price_of_house_with_most_711}")

Price of the house with the most 7-11s within 1km: 0.048431480462300495


In [402]:
# prompt: # prompt: data_house columns ที่มีคำว่า sub_district ทุกอันรวมกัน และแยกชื่อของ sub_district มาเป้นค่าใน row เช่น sub_district_ในเมือง ชื่อ column คือ sub_district ค่าคือ ในเมือง

# Assuming data_house DataFrame is already created and processed as in the provided code.

# Create a new DataFrame to store sub_district information
sub_district_df = pd.DataFrame({'id': data_house['id']})

# Iterate through columns related to sub_district
sub_district_cols = [col for col in data_house.columns if 'sub_district_' in col]
for col in sub_district_cols:
    sub_district_name = col.replace('sub_district_', '')
    sub_district_df[sub_district_name] = data_house[col]
    data_house = data_house.drop(col, axis=1)

# Melt the sub_district DataFrame to the desired format
sub_district_df = sub_district_df.melt(id_vars=['id'], var_name='sub_district', value_name='value')
sub_district_df = sub_district_df[sub_district_df['value'] == 1]

# Merge the sub_district information back to the original DataFrame
data_house = pd.merge(data_house, sub_district_df[['id', 'sub_district']], on='id', how='left')

data_house.head()

<ipython-input-402-17b7859429b1>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sub_district_df[sub_district_name] = data_house[col]
<ipython-input-402-17b7859429b1>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sub_district_df[sub_district_name] = data_house[col]
<ipython-input-402-17b7859429b1>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a

,id,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,usable_area,number_of_parking,sell_price,...,district_โนนศิลา,district_ในเมือง,renovate_yes,renovate_ไม่ระบุ,d1000-7/11,d1000-school,estimator,score,zone,sub_district
0,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,-1,-1,600000.0,4.0,NaN,ศรีบุญเรือง
1,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,-1,-1,600000.0,4.0,NaN,ศรีบุญเรือง
2,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,-1,-1,1300000.0,4.0,NaN,ศรีบุญเรือง
3,1,0.0,0.0,45.0,0.571429,0.266667,0.4,0.519516,0.4,0.048431,...,False,False,False,False,-1,-1,1300000.0,4.0,NaN,ศรีบุญเรือง
4,4,0.0,0.0,88.0,0.285714,0.133333,0.0,0.323562,0.6,0.102524,...,False,False,False,False,-1,-1,750000.0,3.0,NaN,ทรายมูล


In [403]:
# prompt: ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด

# Calculate the average 'score' for each sub_district
avg_score_by_sub_district = data_house.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest average score
sub_district_with_highest_avg_score = avg_score_by_sub_district.idxmax()

# Print the result
print(f"ตำบลที่มีคะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด: {sub_district_with_highest_avg_score}")

ตำบลที่มีคะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด: ศรีบุญเรือง


# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
จากผลวิเคราะห์ ตอบคำถามได้ดังนี้
1. zone ที่มีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด คือ สามเหลี่ยม-กังสดาล-ม.ขอนแก่น
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด 0.048431480462300495
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด คือ ศรีบุญเรือง